In [1]:
from datetime import datetime
from tkinter import messagebox
import tkinter as tk
import pandas as pd
import locale
import warnings
import os

locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
warnings.filterwarnings("ignore")

In [2]:
def get_file_name() -> list[str]:
    file_list = []
    for file in os.listdir('bases/pagos/'):
        if file.endswith('.xlsx') and 'Base Pagos' in file:
            file_list.append(file)
    return file_list

def get_last_date(dates_list: list[datetime]) -> datetime:
    last_date = dates_list[0]
    for date in dates_list:
        if date > last_date:
            last_date = date
    return last_date

def get_date(file_list: list[str]) -> str:
    dates_list = []
    for file in file_list:
        fecha_list =  file.split(' ')[-1].split('.')[:3]
        dates_list.append(datetime.strptime(fecha_list[0] + '.' + fecha_list[1] + '.' + fecha_list[2], '%d.%m.%Y'))
    return get_last_date(dates_list).strftime('%d.%m.%Y')

fecha = get_date(get_file_name())
fecha

'16.09.2024'

In [3]:
def validar_o_crear_carpeta() -> str:
    fecha_hoy = datetime.today().strftime('%Y.%m.%d')
    
    carpeta_reporte = 'reporte'
    
    carpeta_hoy = os.path.join(carpeta_reporte, fecha_hoy)
    
    if not os.path.exists(carpeta_hoy):
        os.makedirs(carpeta_hoy)
        print(f'Carpeta creada: {carpeta_hoy}')
    else:
        print(f'La carpeta ya existe: {carpeta_hoy}')
    
    if not os.path.exists(carpeta_hoy+'/multi_agencias'):
        os.makedirs(carpeta_hoy+'/multi_agencias')
    else:
        print(f'La carpeta ya existe: {carpeta_hoy}\multi_agencias')
    
    return carpeta_hoy

file_path = validar_o_crear_carpeta()
file_path

Carpeta creada: reporte\2024.09.17


'reporte\\2024.09.17'

In [4]:
base_pagos_path = 'bases/pagos/Base Pagos ' + fecha + '.xlsx'
asignacion_path = 'bases/asignacion/base_asignacion.xlsx'

print(base_pagos_path)
print(asignacion_path)

monoproducto = file_path + '/MONOPRODUCTO ' + fecha + '.xlsx'
multiproducto = file_path + '/MULTIPRODUCTO ' + fecha + '.xlsx'
reactiva = file_path + '/REACTIVA ' + fecha + '.xlsx'
no_enviados = file_path + '/NO ENVIADOS ' + fecha + '.xlsx'
si_enviados = file_path + '/ENVIADOS ' + fecha + '.xlsx'

mono_path = os.path.abspath(monoproducto)
multi_path = os.path.abspath(multiproducto)
react_path = os.path.abspath(reactiva)
no_enviados_path = os.path.abspath(no_enviados)
enviados_path = os.path.abspath(si_enviados)

print(mono_path)
print(multi_path)
print(react_path)
print(no_enviados_path)
print(enviados_path)

bases/pagos/Base Pagos 16.09.2024.xlsx
bases/asignacion/base_asignacion.xlsx
c:\Users\miria\Documents\BBVA\GitHub\base-pagos\reporte\2024.09.17\MONOPRODUCTO 16.09.2024.xlsx
c:\Users\miria\Documents\BBVA\GitHub\base-pagos\reporte\2024.09.17\MULTIPRODUCTO 16.09.2024.xlsx
c:\Users\miria\Documents\BBVA\GitHub\base-pagos\reporte\2024.09.17\REACTIVA 16.09.2024.xlsx
c:\Users\miria\Documents\BBVA\GitHub\base-pagos\reporte\2024.09.17\NO ENVIADOS 16.09.2024.xlsx
c:\Users\miria\Documents\BBVA\GitHub\base-pagos\reporte\2024.09.17\ENVIADOS 16.09.2024.xlsx


In [5]:
#################### BASE DE PAGOS ####################

In [6]:
df_base = pd.read_excel(base_pagos_path)
print(df_base.shape)
print(df_base.dtypes)
df_base.head(5)

(371, 9)
Entidad         int64
FECHA          object
CODCEN          int64
CLAVSERV        int64
CENTROPAGO      int64
IMPORTE       float64
MONEDA         object
NOMBRE         object
ESTADO         object
dtype: object


,Entidad,FECHA,CODCEN,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO
0,11,2024-09-16,24460523,248,128,1320.0,PEN,SEGUNDO SANTOS AGUILAR NOVOA,P
1,11,2024-09-16,21883058,248,30,7500.0,PEN,D' BALL EIRL,P
2,11,2024-09-16,29254244,248,174,6750.0,PEN,GRUPO CORPORATIVO LICIADE E.I.R.L,P
3,11,2024-09-14,31388955,248,314,2050.0,PEN,..,P
4,11,2024-09-16,20089973,248,273,800.0,PEN,",,",P


In [7]:
fecha_formateada = pd.to_datetime('today').strftime('%d-%b')
fecha_formateada = fecha_formateada[:3] + fecha_formateada[3:].capitalize()

df_base['FECHA_ENVIO'] = fecha_formateada.replace('.', '')
df_base['ID_RESPONSABLE'] = 'MIV'

cols_base = ['FECHA', 'CODCEN', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FECHA_ENVIO', 'ID_RESPONSABLE']
df_base = df_base[cols_base]
df_base = df_base.rename(columns={'CODCEN': 'CC'})

df_base['CC'] = df_base['CC'].astype('Int64').astype(str).str.zfill(8)
df_base['CC'] = df_base['CC'].str.replace(' ', '').str.replace(r'\D', '', regex=True).str[-8:]
df_base['CC'] = df_base['CC'].str.zfill(8)

df_base['CLAVSERV'] = df_base['CLAVSERV'].astype(str).str.zfill(5)

print(df_base.shape)
print(df_base.dtypes)
df_base.head(5)

(371, 10)
FECHA              object
CC                 object
CLAVSERV           object
CENTROPAGO          int64
IMPORTE           float64
MONEDA             object
NOMBRE             object
ESTADO             object
FECHA_ENVIO        object
ID_RESPONSABLE     object
dtype: object


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FECHA_ENVIO,ID_RESPONSABLE
0,2024-09-16,24460523,00248,128,1320.0,PEN,SEGUNDO SANTOS AGUILAR NOVOA,P,17-Sep,MIV
1,2024-09-16,21883058,00248,30,7500.0,PEN,D' BALL EIRL,P,17-Sep,MIV
2,2024-09-16,29254244,00248,174,6750.0,PEN,GRUPO CORPORATIVO LICIADE E.I.R.L,P,17-Sep,MIV
3,2024-09-14,31388955,00248,314,2050.0,PEN,..,P,17-Sep,MIV
4,2024-09-16,20089973,00248,273,800.0,PEN,",,",P,17-Sep,MIV


In [8]:
df_base.isnull().sum()

FECHA             0
CC                0
CLAVSERV          0
CENTROPAGO        0
IMPORTE           0
MONEDA            0
NOMBRE            0
ESTADO            1
FECHA_ENVIO       0
ID_RESPONSABLE    0
dtype: int64

In [9]:
#################### ASIGNACION ####################

In [10]:
root = tk.Tk()
root.attributes("-topmost", True)
root.withdraw()

result = messagebox.askquestion("Confirmación", "¿Cargar base de asignación?", icon='warning')
if result == 'yes':
    df_asignacion = pd.read_excel(asignacion_path)

In [11]:
print(df_asignacion.shape)
print(df_asignacion.dtypes)
df_asignacion.head(5)

(240733, 8)
codigo               int64
ID_FLAG              int64
cont_18              int64
nombre              object
TIPO_CARTERA        object
tipo_fondo          object
clave               object
AGENCIA CORRECTA    object
dtype: object


,codigo,ID_FLAG,cont_18,nombre,TIPO_CARTERA,tipo_fondo,clave,AGENCIA CORRECTA
0,2569,1,1101075001274140,GLADYS MERELYN ABANTO GONZALES,UNSECURED,NaN,KSTBC,ASESCOM RJ
1,14583,2,1101235001222660,ADMINISTRACION DE EMPRESAS LIBRERIA EDITORIAL SAC,UNSECURED,FAE,BC,SIN AGENCIA
2,14583,2,1101235001218442,ADMINISTRACION DE EMPRESAS LIBRERIA EDITORIAL SAC,UNSECURED,FAE,BC,SIN AGENCIA
3,16896,1,1101305001000046,PABLO AGARIE MIYASATO,UNSECURED,NaN,KSTBC,ASESCOM RJ
4,17507,1,1108098100538643,AGENCIA DE ADUANA YVAPORES LABARTHE SA,SECURED,NaN,BC,SIN AGENCIA


In [12]:
#################### BACKUPS ####################

In [13]:
df_base_test = df_base.copy()
print('Base', df_base_test.shape)

df_asignacion_test = df_asignacion.copy()
print('Asignacion', df_asignacion_test.shape)

Base (371, 10)
Asignacion (240733, 8)


In [14]:
base_count = df_base_test.shape[0]
base_count

371

In [15]:
def actualizar_tipo_cartera(grupo: pd.DataFrame) -> pd.DataFrame:
    if 'UNSECURED' in grupo['TIPO_CARTERA'].values:
        grupo['TIPO_CARTERA'] = 'UNSECURED'
    return grupo

In [16]:
df_asignacion_test['codigo'] = df_asignacion_test['codigo'].astype('Int64').astype(str).str.zfill(8)

df_asignacion_test = df_asignacion_test.rename(columns={'tipo_fondo': 'TIPO_FONDO'})
cols_asignacion = ['codigo', 'ID_FLAG', 'cont_18', 'nombre', 'TIPO_CARTERA', 'TIPO_FONDO', 'clave', 'AGENCIA CORRECTA']
df_base_test = df_base_test.merge(df_asignacion_test[cols_asignacion], left_on='CC', right_on='codigo', how='left')
df_base_test = df_base_test.drop(columns=['codigo'])
df_base_test = df_base_test.rename(columns={'ID_FLAG': 'FLAG', 'cont_18': 'CONTRATO', 'nombre': 'NOMBRE_CLIENTE', 'clave': 'CARTERA', 'AGENCIA CORRECTA': 'AGENCIA'})
df_base_test.sort_values(by=['TIPO_CARTERA'], ascending=False, inplace=True)
df_base_test.sort_values(by=['CC'], ascending=True, inplace=True)

filtro = (df_base_test['FLAG'] != 1) & (df_base_test['FLAG'].notna())
df_base_filtro = df_base_test.loc[filtro].groupby('CC').apply(actualizar_tipo_cartera).reset_index(drop=True)
df_base_test = df_base_test.merge(df_base_filtro[['CC', 'TIPO_CARTERA']], on='CC', how='left', suffixes=('', '_y'))
df_base_test['TIPO_CARTERA_FINAL'] = df_base_test['TIPO_CARTERA_y'].fillna(df_base_test['TIPO_CARTERA'])
df_base_test = df_base_test.drop(columns=['TIPO_CARTERA', 'TIPO_CARTERA_y'])
df_base_test = df_base_test.rename(columns={'TIPO_CARTERA_FINAL': 'TIPO_CARTERA'})
#26497246
df_base_test.drop_duplicates(subset=['CC', 'IMPORTE', 'MONEDA', 'NOMBRE'], keep='first', inplace=True)

df_base_test['TIPO_CARTERA'] = df_base_test['TIPO_CARTERA'].fillna('NULL')
df_base_test['TIPO_FONDO'] = df_base_test['TIPO_FONDO'].fillna('NULL')

df_base_test['FLAG'] = df_base_test['FLAG'].astype('Int64')
df_base_test['CONTRATO'] = df_base_test['CONTRATO'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)

cols_base_test = ['FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'AGENCIA']
df_base_test = df_base_test[cols_base_test]

print(df_base_test.shape)
print(df_base_test.dtypes)
df_base_test.head(5)

(371, 17)
FECHA              object
CC                 object
CLAVSERV           object
CENTROPAGO          int64
IMPORTE           float64
MONEDA             object
NOMBRE             object
ESTADO             object
FLAG                Int64
CONTRATO           object
TIPO_FONDO         object
CARTERA            object
NOMBRE_CLIENTE     object
FECHA_ENVIO        object
ID_RESPONSABLE     object
TIPO_CARTERA       object
AGENCIA            object
dtype: object


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
0,2024-09-16,01162268,00248,118,1700.0,PEN,ROSA LITA MANTILLA MORALES,P,1,001108319601385258,NULL,EXJ,ROSA LITA MANTILLA MORALES,17-Sep,MIV,UNSECURED,CLASA MORA
1,2024-09-16,04893700,00248,960,265.0,PEN,JOSE JULIO GARAY DUE･AS,P,1,001101405002103900,NULL,KSTBC,JOSE JULIO GARAY DUEÐAS,17-Sep,MIV,UNSECURED,MORNESE MORA
2,2024-09-16,05847052,00248,597,100.0,PEN,TARJETA,P,1,001105975000325936,NULL,KSTBC,MARIA VICTORIA PEREZ SOLF,17-Sep,MIV,UNSECURED,ASESCOM RJ
3,2024-09-16,07007612,00248,190,300.0,PEN,CARLOS ANTONIO TORRES FLORES 1,P,1,001101005001032297,NULL,KSTBC,CARLOS ANTONIO TORRES FLORES,17-Sep,MIV,UNSECURED,ASESCOM RJ
4,2024-09-16,07365110,00248,228,1000.0,PEN,",DOMINGO LIMA MAMANI",P,1,001102289600369463,NULL,KSTBC,DOMINGO LIMA MAMANI,17-Sep,MIV,UNSECURED,CLASA MORA


In [17]:
if base_count == df_base_test.shape[0]:
    print('REGISTROS OK')
elif base_count > df_base_test.shape[0]:
    print('Se han perdido registros')
else:
    print('Se han duplicado registros: ', df_base_test.shape[0] - base_count)

REGISTROS OK


In [18]:
df_base_test.shape

(371, 17)

In [19]:
df_base_no_flag = df_base_test[df_base_test['FLAG'].isnull()]
df_base_no_flag.shape

(11, 17)

In [20]:
no_flag_count = df_base_no_flag.shape[0]
no_flag_count

11

In [21]:
flag_count = base_count - no_flag_count
flag_count

360

In [22]:
#################### MONOPRODUCTO ####################

In [23]:
df_mono = df_base_test[df_base_test['FLAG'] == 1]
print(df_mono.shape)
df_mono.head(5)

(305, 17)


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
0,2024-09-16,01162268,00248,118,1700.0,PEN,ROSA LITA MANTILLA MORALES,P,1,001108319601385258,NULL,EXJ,ROSA LITA MANTILLA MORALES,17-Sep,MIV,UNSECURED,CLASA MORA
1,2024-09-16,04893700,00248,960,265.0,PEN,JOSE JULIO GARAY DUE･AS,P,1,001101405002103900,NULL,KSTBC,JOSE JULIO GARAY DUEÐAS,17-Sep,MIV,UNSECURED,MORNESE MORA
2,2024-09-16,05847052,00248,597,100.0,PEN,TARJETA,P,1,001105975000325936,NULL,KSTBC,MARIA VICTORIA PEREZ SOLF,17-Sep,MIV,UNSECURED,ASESCOM RJ
3,2024-09-16,07007612,00248,190,300.0,PEN,CARLOS ANTONIO TORRES FLORES 1,P,1,001101005001032297,NULL,KSTBC,CARLOS ANTONIO TORRES FLORES,17-Sep,MIV,UNSECURED,ASESCOM RJ
4,2024-09-16,07365110,00248,228,1000.0,PEN,",DOMINGO LIMA MAMANI",P,1,001102289600369463,NULL,KSTBC,DOMINGO LIMA MAMANI,17-Sep,MIV,UNSECURED,CLASA MORA


In [24]:
mono_count = df_mono.shape[0]
mono_count

305

In [25]:
df_mono['TIPO_CARTERA'].value_counts()

TIPO_CARTERA
UNSECURED    299
SECURED        6
Name: count, dtype: int64

In [26]:
df_mono['TIPO_FONDO'].value_counts()

TIPO_FONDO
NULL        298
REACTIVA      6
CONVENIO      1
Name: count, dtype: int64

In [27]:
df_mono[['TIPO_CARTERA', 'TIPO_FONDO']].value_counts()

TIPO_CARTERA  TIPO_FONDO
UNSECURED     NULL          296
SECURED       REACTIVA        4
              NULL            2
UNSECURED     REACTIVA        2
              CONVENIO        1
Name: count, dtype: int64

In [28]:
#################### MONO REACTIVA ####################

In [29]:
df_reactiva = df_mono[(df_mono['TIPO_CARTERA'] == 'UNSECURED') & (df_mono['TIPO_FONDO'] != 'NULL')]
df_reactiva.sort_values(by=['CC'], inplace=True)
df_reactiva.reset_index(drop=True, inplace=True)
print(df_reactiva.shape)
df_reactiva.head(5)

(3, 17)


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
0,2024-09-16,25474777,00248,332,4000.0,PEN,JHENNY CARMEN RODRIGUEZ AMARO.,P,1,001101839600423847,REACTIVA,KSTBC,JHENNY CARMEN RODRIGUEZ AMARO,17-Sep,MIV,UNSECURED,MORNESE MORA
1,2024-09-14,29192668,00248,257,4419.0,PEN,PERCY MULLO ROJAS,P,1,001101119600222202,REACTIVA,KSTBC,PERCY MULLO ROJAS,17-Sep,MIV,UNSECURED,MORNESE MORA
2,2024-09-16,30487727,00248,162,43923.6,PEN,BRYAN ARTURO DAVILA GONZALEZ,P,1,001107649600097417,CONVENIO,EXJ,BRYAN ARTURO DAVILA GONZALEZ,17-Sep,MIV,UNSECURED,SIN AGENCIA


In [30]:
reactiva_count = df_reactiva.shape[0]
reactiva_count

3

In [31]:
df_reactiva.to_excel(reactiva, index=False)

In [32]:
#################### FIN REACTIVA ####################

In [33]:
df_no_mono = df_mono[(df_mono['TIPO_CARTERA'] != 'UNSECURED')]
no_mono_count = df_no_mono.shape[0]
no_mono_count

6

In [34]:
df_mono = df_mono[df_mono['TIPO_CARTERA'] == 'UNSECURED']
df_mono = df_mono[df_mono['TIPO_FONDO'] == 'NULL']

cols_mono = ['FECHA', 'CC', 'CLAVSERV', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'CONTRATO', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'TIPO_FONDO', 'CARTERA', 'AGENCIA']
df_mono = df_mono[cols_mono]

df_mono.sort_values(by=['CC'], inplace=True)
df_mono.reset_index(drop=True, inplace=True)

print(df_mono.shape)
df_mono.head(5)

(296, 16)


,FECHA,CC,CLAVSERV,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,TIPO_FONDO,CARTERA,AGENCIA
0,2024-09-16,01162268,00248,1700.0,PEN,ROSA LITA MANTILLA MORALES,P,1,001108319601385258,ROSA LITA MANTILLA MORALES,17-Sep,MIV,UNSECURED,NULL,EXJ,CLASA MORA
1,2024-09-16,04893700,00248,265.0,PEN,JOSE JULIO GARAY DUE･AS,P,1,001101405002103900,JOSE JULIO GARAY DUEÐAS,17-Sep,MIV,UNSECURED,NULL,KSTBC,MORNESE MORA
2,2024-09-16,05847052,00248,100.0,PEN,TARJETA,P,1,001105975000325936,MARIA VICTORIA PEREZ SOLF,17-Sep,MIV,UNSECURED,NULL,KSTBC,ASESCOM RJ
3,2024-09-16,07007612,00248,300.0,PEN,CARLOS ANTONIO TORRES FLORES 1,P,1,001101005001032297,CARLOS ANTONIO TORRES FLORES,17-Sep,MIV,UNSECURED,NULL,KSTBC,ASESCOM RJ
4,2024-09-16,07365110,00248,1000.0,PEN,",DOMINGO LIMA MAMANI",P,1,001102289600369463,DOMINGO LIMA MAMANI,17-Sep,MIV,UNSECURED,NULL,KSTBC,CLASA MORA


In [35]:
df_mono.to_excel(monoproducto, index=False)

In [36]:
mono_count_final = df_mono.shape[0]
mono_count_final

296

In [37]:
if mono_count == mono_count_final + reactiva_count + no_mono_count:
    print('MONOPRODUCTO OK')
else:
    print('MONOPRODUCTO ERROR')

MONOPRODUCTO OK


In [38]:
#################### FIN MONOPRODUCTO ####################

In [39]:
#################### MULTIPRODUCTO ####################

In [40]:
df_multi = df_base_test[df_base_test['FLAG'] > 1].copy()
print(df_multi.shape)
df_multi.head(5)

(55, 17)


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
5,2024-09-16,08434980,00248,259,360.5,PEN,YOLANDA ESPERANZA TONG RONDONA DE MEDINA,P,2,001107855003847218,NULL,KSTBC,YOLANDA ESPERANZA YONG RONDONA,17-Sep,MIV,UNSECURED,MORNESE MORA
14,2024-09-16,12322224,00248,317,1000.0,PEN,"PAREDES BRICE･O GENNO GUILLERMO,",P,2,001104085011567010,NULL,EXJ,GUILLERMO PAREDES BRICEÐO GENNO,17-Sep,MIV,UNSECURED,SIN AGENCIA
36,2024-09-16,20572603,00248,370,1500.0,PEN,OPTICAL & PLUS CORP SRL,P,5,001101615001510701,NULL,BC,OPTICAL & PLUS CORPSRL,17-Sep,MIV,UNSECURED,RodriguezAngobaldo
61,2024-09-16,20597258,00248,125,1000.0,PEN,ANA MARIA LARA OTAZU 10340814,P,2,001102619600041700,NULL,EXJ,ANA MARIA LARA OTAZU,17-Sep,MIV,UNSECURED,ASESCOM RJ
68,2024-09-16,20756586,00248,223,1231.0,PEN,CLAVER SERGIO TITO QUISPE,P,2,001108319601346724,NULL,KSTBC,CLAVER SERGIO TITO QUISPE,17-Sep,MIV,UNSECURED,CLASA MORA


In [41]:
multi_count = df_multi.shape[0]
multi_count

55

In [42]:
df_multi['TIPO_CARTERA'].value_counts()

TIPO_CARTERA
UNSECURED    52
SECURED       3
Name: count, dtype: int64

In [43]:
df_multi['TIPO_FONDO'].value_counts()

TIPO_FONDO
NULL        51
REACTIVA     4
Name: count, dtype: int64

In [44]:
df_no_multi = df_multi[(df_multi['TIPO_CARTERA'] != 'UNSECURED')]
no_multi_count = df_no_multi.shape[0]
no_multi_count

3

In [45]:
df_multi = df_multi[df_multi['TIPO_CARTERA'] == 'UNSECURED']

df_multi['CONTRATO'] = None
df_multi['TIPO_FONDO'] = None
df_multi['CARTERA'] = None

cols_multi = ['FECHA', 'CC', 'CLAVSERV', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE']
df_multi = df_multi[cols_multi]

df_multi = df_multi.drop_duplicates(subset=['CC', 'IMPORTE', 'MONEDA', 'NOMBRE'])

df_multi.sort_values(by=['CC'], inplace=True)
df_multi.reset_index(drop=True, inplace=True)

print(df_multi.shape)
df_multi.head(5)

(52, 14)


,FECHA,CC,CLAVSERV,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE
0,2024-09-16,08434980,00248,360.5,PEN,YOLANDA ESPERANZA TONG RONDONA DE MEDINA,P,2,None,None,None,YOLANDA ESPERANZA YONG RONDONA,17-Sep,MIV
1,2024-09-16,12322224,00248,1000.0,PEN,"PAREDES BRICE･O GENNO GUILLERMO,",P,2,None,None,None,GUILLERMO PAREDES BRICEÐO GENNO,17-Sep,MIV
2,2024-09-16,20572603,00248,1500.0,PEN,OPTICAL & PLUS CORP SRL,P,5,None,None,None,OPTICAL & PLUS CORPSRL,17-Sep,MIV
3,2024-09-16,20597258,00248,1000.0,PEN,ANA MARIA LARA OTAZU 10340814,P,2,None,None,None,ANA MARIA LARA OTAZU,17-Sep,MIV
4,2024-09-16,20756586,00248,1231.0,PEN,CLAVER SERGIO TITO QUISPE,P,2,None,None,None,CLAVER SERGIO TITO QUISPE,17-Sep,MIV


In [46]:
df_multi.to_excel(multiproducto, index=False)

In [47]:
multi_count_final = df_multi.shape[0]
multi_count_final

52

In [48]:
if multi_count == multi_count_final + no_multi_count:
    print('MULTIPRODUCTO OK')
else:
    print('MULTIPRODUCTO ERROR')

MULTIPRODUCTO OK


In [49]:
#################### FIN MULTIPRODUCTO ####################

In [50]:
#################### NO ENVIADOS ####################

In [51]:
no_eviados = no_mono_count + no_multi_count + no_flag_count
enviados = mono_count_final + multi_count_final + reactiva_count

print('No enviados:', no_eviados)
print('Enviados:', enviados)
print('Total:', no_eviados + enviados)

No enviados: 20
Enviados: 351
Total: 371


In [52]:
if base_count == enviados + no_eviados:
    print('REGISTROS OK')
else:
    print('REGISTROS ERROR')

REGISTROS OK


In [53]:
print(df_no_mono.columns)
print(df_no_multi.columns)
print(df_base_no_flag.columns)

Index(['FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE',
       'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE',
       'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'AGENCIA'],
      dtype='object')
Index(['FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE',
       'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE',
       'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'AGENCIA'],
      dtype='object')
Index(['FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE',
       'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE',
       'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'AGENCIA'],
      dtype='object')


In [54]:
df_no_enviados = pd.concat([df_no_mono, df_no_multi, df_base_no_flag])
df_no_enviados['FLAG'] = df_no_enviados['FLAG'].astype('Int64').fillna(0)

df_no_enviados['CONTRATO'] = df_no_enviados.apply(lambda x: None if x['FLAG'] != 1 else x['CONTRATO'], axis=1)
df_no_enviados['TIPO_FONDO'] = df_no_enviados.apply(lambda x: None if x['FLAG'] != 1 else x['TIPO_FONDO'], axis=1)
df_no_enviados['TIPO_CARTERA'] = df_no_enviados.apply(lambda x: None if x['FLAG'] != 1 else x['TIPO_CARTERA'], axis=1)

df_no_enviados['FLAG'] = df_no_enviados['FLAG'].apply(lambda x: '' if x == 0 else x)

df_no_enviados.sort_values(by=['CC'], inplace=True)
df_no_enviados.reset_index(drop=True, inplace=True)

print(df_no_enviados.shape)
print(df_no_enviados.dtypes)
df_no_enviados.head(5)

(20, 17)
FECHA              object
CC                 object
CLAVSERV           object
CENTROPAGO          int64
IMPORTE           float64
MONEDA             object
NOMBRE             object
ESTADO             object
FLAG               object
CONTRATO           object
TIPO_FONDO         object
CARTERA            object
NOMBRE_CLIENTE     object
FECHA_ENVIO        object
ID_RESPONSABLE     object
TIPO_CARTERA       object
AGENCIA            object
dtype: object


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
0,2024-09-16,10569370,00248,338,11772.00,PEN,LILIANA GIOVANNA BERROCAL BERRIOS,P,,None,None,NaN,NaN,17-Sep,MIV,None,NaN
1,2024-09-16,20012314,00248,23,1075.00,PEN,CIRILA GAUDIOSA VERA ACOSTA DNI 07553798,P,,None,None,NaN,NaN,17-Sep,MIV,None,NaN
2,2024-09-16,20956913,00248,245,5000.00,PEN,JIMMY SERRANO VILLALTA.,P,,None,None,NaN,NaN,17-Sep,MIV,None,NaN
3,2024-09-16,21117543,00248,259,2665.23,PEN,KENY NIEVES HUAMANI JIMENEZ,P,,None,None,NaN,NaN,17-Sep,MIV,None,NaN
4,2024-09-16,21212725,00248,175,35813.16,PEN,JANET ESTHER LEIVA MEZA,P,1,001101629600205667,NULL,BC,JANET ESTHER LEIVA MEZA,17-Sep,MIV,SECURED,SIN AGENCIA


In [55]:
if no_eviados == df_no_enviados.shape[0]:
    print('NO ENVIADOS OK')
else:
    print('NO ENVIADOS ERROR')

NO ENVIADOS OK


In [56]:
df_no_enviados.to_excel(no_enviados, index=False)

In [57]:
#################### FORMATO EXCEL ####################

In [58]:
import openpyxl as op
from openpyxl.styles import Font, PatternFill, Alignment

def format_excel(file_path: str, validator: str) -> None:
    workbook = op.load_workbook(file_path)
    sheet = workbook.active
    
    # Definir estilos generales
    general_font = Font(name='Calibri', size=11)
    header_font = Font(name='Calibri', size=11, bold=True, color='FFFFFF')
    alignment_center = Alignment(horizontal='center', vertical='center')
    
    # Definir estilos específicos
    header_font_white = Font(name='Calibri', size=11, bold=True, color='000000')
    header_fill_blue = PatternFill(start_color='002060', end_color='002060', fill_type='solid')
    header_fill_yellow = PatternFill(start_color='FFD965', end_color='FFD965', fill_type='solid')
    header_fill_green = PatternFill(start_color='C4D79B', end_color='C4D79B', fill_type='solid')
    header_fill_orange = PatternFill(start_color='FABF8F', end_color='FABF8F', fill_type='solid')
    
    # Aplicar estilos generales a todas las celdas
    for row in sheet.iter_rows():
        for cell in row:
            cell.font = general_font
    
    # Aplicar estilos al encabezado (fila 1)
    for cell in sheet[1]:
        cell.font = header_font
        cell.alignment = alignment_center
    
    # Aplicar estilos específicos según el validador
    if validator == 'mono':
        for col in range(1, 10):  # Columnas A-I
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(10, 14):  # Columnas J-M
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(14, 16):  # Columnas N-O
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(16, 17):  # Columna P
            sheet.cell(row=1, column=col).fill = header_fill_orange
            sheet.cell(row=1, column=col).font = header_font_white
    
    elif validator == 'multi':
        for col in range(1, 10):  # Columnas A-I
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(10, 12):  # Columnas J-K
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(12, 15):  # Columnas L-N
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
    
    elif validator == 'multi_final':
        for col in range(1, 10):  # Columnas A-I
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(10, 12):  # Columnas J-K
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(12, 16):  # Columnas L-P
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(16, 17):  # Columna P
            sheet.cell(row=1, column=col).fill = header_fill_orange
            sheet.cell(row=1, column=col).font = header_font_white
    
    elif validator == 'instruccion':
        for col in range(1, 11): # Columnas A-J
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(11, 13): # Columnas K-L
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(13, 17):  # Columnas M-P
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(17, 18):  # Columna Q
            sheet.cell(row=1, column=col).fill = header_fill_orange
            sheet.cell(row=1, column=col).font = header_font_white
    
    workbook.save(file_path)

In [59]:
format_excel(mono_path, "mono")
format_excel(multi_path, "multi")
format_excel(react_path, "instruccion")
format_excel(no_enviados_path, "instruccion")

In [60]:
os.startfile(mono_path)
os.startfile(multi_path)
os.startfile(react_path)        
os.startfile(no_enviados_path)

In [61]:
os.startfile(react_path)  

In [62]:
#################### AGENCIAS ####################

In [63]:
x

NameError: name 'x' is not defined

In [64]:
df_1 = pd.read_excel(file_path+'/multi_agencias/RJ.xlsx')
df_2 = pd.read_excel(file_path+'/multi_agencias/CLASA.xlsx')
df_3 = pd.read_excel(file_path+'/multi_agencias/MORNESE.xlsx')


df_1 = df_1[cols_multi]
df_2 = df_2[cols_multi]
df_3 = df_3[cols_multi]

df_1['AGENCIA'] = 'ASESCOM RJ'
df_2['AGENCIA'] = 'CLASA MORA'
df_3['AGENCIA'] = 'MORNESE MORA'

print(df_1.shape)
print(df_2.shape)
print(df_3.shape)

(51, 15)
(51, 15)
(53, 15)


In [65]:
print("Contratos RJ:", df_1['CONTRATO'].notna().value_counts()[1] - 1)
print("Contratos CLASA:", df_2['CONTRATO'].notna().value_counts()[1] - 1)
print("Contratos MORNESE:", df_3['CONTRATO'].notna().value_counts()[1] - 1)
print("Total:", df_1['CONTRATO'].notna().value_counts()[1] + df_2['CONTRATO'].notna().value_counts()[1] + df_3['CONTRATO'].notna().value_counts()[1] - 3)

Contratos RJ: 17
Contratos CLASA: 17
Contratos MORNESE: 16
Total: 50


In [66]:
df_agencias = pd.concat([df_1, df_2, df_3])
df_agencias.dropna(subset=['CONTRATO'], inplace=True)
df_agencias['CONTRATO'] = df_agencias['CONTRATO'].apply(lambda x: str(int(x)).zfill(18) if len(x)==18 else None)
df_agencias.dropna(subset=['CONTRATO'], inplace=True)
df_agencias.reset_index(drop=True, inplace=True)

print(df_agencias.shape)
print(df_agencias.dtypes)
df_agencias.head(5)

(50, 15)
FECHA              object
CC                  int64
CLAVSERV            int64
IMPORTE           float64
MONEDA             object
NOMBRE             object
ESTADO             object
FLAG                int64
CONTRATO           object
TIPO_FONDO         object
CARTERA            object
NOMBRE_CLIENTE     object
FECHA_ENVIO        object
ID_RESPONSABLE     object
AGENCIA            object
dtype: object


,FECHA,CC,CLAVSERV,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,AGENCIA
0,2024-09-16,20597258,248,1000.0,PEN,ANA MARIA LARA OTAZU 10340814,P,2,001102619600041700,NaN,EXJ,ANA MARIA LARA OTAZU,17-Sep,MIV,ASESCOM RJ
1,2024-09-16,24272121,248,2320.0,PEN,KELYN LILIANA PAREDES RIOS,P,2,001109005001005051,NaN,EXJ,KELYN LILIANA PAREDES RIOS,17-Sep,MIV,ASESCOM RJ
2,2024-09-16,25560582,248,500.0,PEN,INVERSIONES GENERALES ANDALUCIA EIRL,P,6,001104379601966887,NaN,BC,INVERSIONES GENERALES ANDALUCIA EIRL,17-Sep,MIV,ASESCOM RJ
3,2024-09-16,25879951,248,700.0,PEN,JAINOR ABRAHAM ESPINOZA GUTIERREZ,P,2,001105045001623739,NaN,KSTBC,JAINOR ABRAHAM ESPINOZA GUTIERREZ,17-Sep,MIV,ASESCOM RJ
4,2024-09-14,26208379,248,1200.0,PEN,LUBRICANTES LUISA SAC/ 26208379,P,2,001104379601505901,NaN,BC,LUBRICANTES LUISA SAC,17-Sep,MIV,ASESCOM RJ


In [67]:
df_multi = pd.read_excel(multi_path)
df_multi['AGENCIA'] = 'NULL'
print(df_multi.shape)
df_multi.head(5)

(51, 15)


,FECHA,CC,CLAVSERV,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,AGENCIA
0,2024-09-16,8434980,248,360.5,PEN,YOLANDA ESPERANZA TONG RONDONA DE MEDINA,P,2,NaN,NaN,NaN,YOLANDA ESPERANZA YONG RONDONA,17-Sep,MIV,NULL
1,2024-09-16,12322224,248,1000.0,PEN,"PAREDES BRICE･O GENNO GUILLERMO,",P,2,NaN,NaN,NaN,GUILLERMO PAREDES BRICEÐO GENNO,17-Sep,MIV,NULL
2,2024-09-16,20572603,248,1500.0,PEN,OPTICAL & PLUS CORP SRL,P,5,NaN,NaN,NaN,OPTICAL & PLUS CORPSRL,17-Sep,MIV,NULL
3,2024-09-16,20597258,248,1000.0,PEN,ANA MARIA LARA OTAZU 10340814,P,2,NaN,NaN,NaN,ANA MARIA LARA OTAZU,17-Sep,MIV,NULL
4,2024-09-16,20756586,248,1231.0,PEN,CLAVER SERGIO TITO QUISPE,P,2,NaN,NaN,NaN,CLAVER SERGIO TITO QUISPE,17-Sep,MIV,NULL


In [68]:
df_multi_final = pd.concat([df_multi, df_agencias])

df_multi_final['CC'] = df_multi_final['CC'].astype('Int64').astype(str).str.zfill(8)
df_multi_final.drop(columns=['TIPO_FONDO', 'CARTERA'], inplace=True)
df_multi_final.sort_values(by=['CC', 'CONTRATO'], inplace=True)

df_multi_final.drop_duplicates(subset=['CC', 'IMPORTE', 'MONEDA', 'NOMBRE'], keep='first', inplace=True)
df_multi_final.reset_index(drop=True, inplace=True)

df_asignacion_test['codigo'] = df_asignacion_test['codigo'].astype('Int64').astype(str).str.zfill(8)
df_asignacion_test['cont_18'] = df_asignacion_test['cont_18'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)

cols_asignacion = ['cont_18', 'TIPO_FONDO', 'clave', 'TIPO_CARTERA', 'AGENCIA CORRECTA']
df_multi_final = df_multi_final.merge(df_asignacion_test[cols_asignacion], left_on='CONTRATO', right_on='cont_18', how='left')

df_multi_final = df_multi_final.drop(columns=['cont_18'])

df_multi_final = df_multi_final.rename(columns={'clave': 'CARTERA'})

df_multi_final['CONTRATO'] = df_multi_final['CONTRATO'].fillna('NULL')
df_multi_final['TIPO_FONDO'] = df_multi_final['TIPO_FONDO'].fillna('NULL')
df_multi_final['CARTERA'] = df_multi_final['CARTERA'].fillna('NULL')
df_multi_final['TIPO_CARTERA'] = df_multi_final['TIPO_CARTERA'].fillna('NULL')
df_multi_final['AGENCIA CORRECTA'] = df_multi_final['AGENCIA CORRECTA'].fillna('NULL')
df_multi_final['CLAVSERV'] = df_multi_final['CLAVSERV'].astype(str).str.zfill(5)

df_multi_final['AGENCIA CORRECTA'] = df_multi_final.apply(lambda row: row['AGENCIA CORRECTA'] if row['AGENCIA CORRECTA'] == row['AGENCIA'] else f"{row['AGENCIA']}/{row['AGENCIA CORRECTA']}", axis=1)

df_multi_final = df_multi_final.drop(columns=['AGENCIA'])
df_multi_final.rename(columns={'AGENCIA CORRECTA': 'AGENCIA'}, inplace=True)

cols_multi_final = ['FECHA', 'CC', 'CLAVSERV', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'AGENCIA']
df_multi_final = df_multi_final[cols_multi_final]
df_multi_final.reset_index(drop=True, inplace=True)

print(df_multi_final.shape)
print(df_multi_final.dtypes)
df_multi_final.head(5)

(53, 16)
FECHA              object
CC                 object
CLAVSERV           object
IMPORTE           float64
MONEDA             object
NOMBRE             object
ESTADO             object
FLAG                int64
CONTRATO           object
TIPO_FONDO         object
CARTERA            object
NOMBRE_CLIENTE     object
FECHA_ENVIO        object
ID_RESPONSABLE     object
TIPO_CARTERA       object
AGENCIA            object
dtype: object


,FECHA,CC,CLAVSERV,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
0,2024-09-16,08434980,00248,360.5,PEN,YOLANDA ESPERANZA TONG RONDONA DE MEDINA,P,2,001107855003847218,NULL,KSTBC,YOLANDA ESPERANZA YONG RONDONA,17-Sep,MIV,UNSECURED,MORNESE MORA
1,2024-09-16,12322224,00248,1000.0,PEN,"PAREDES BRICE･O GENNO GUILLERMO,",P,2,001101199600155894,NULL,EXJ,GUILLERMO PAREDES BRICEÐO GENNO,17-Sep,MIV,UNSECURED,MORNESE MORA
2,2024-09-16,20572603,00248,1500.0,PEN,OPTICAL & PLUS CORP SRL,P,5,NULL,NULL,NULL,OPTICAL & PLUS CORPSRL,17-Sep,MIV,NULL,NULL
3,2024-09-16,20597258,00248,1000.0,PEN,ANA MARIA LARA OTAZU 10340814,P,2,001102619600041700,NULL,EXJ,ANA MARIA LARA OTAZU,17-Sep,MIV,UNSECURED,ASESCOM RJ
4,2024-09-16,20756586,00248,1231.0,PEN,CLAVER SERGIO TITO QUISPE,P,2,001105045002540055,NULL,KSTBC,CLAVER SERGIO TITO QUISPE,17-Sep,MIV,UNSECURED,CLASA MORA


In [69]:
df_no_enviados_multi = df_multi_final[(df_multi_final['CONTRATO'] == 'NULL') | (df_multi_final['TIPO_CARTERA'] != 'UNSECURED')]
df_no_enviados_multi

,FECHA,CC,CLAVSERV,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
2,2024-09-16,20572603,00248,1500.0,PEN,OPTICAL & PLUS CORP SRL,P,5,NULL,NULL,NULL,OPTICAL & PLUS CORPSRL,17-Sep,MIV,NULL,NULL
8,2024-09-16,23108343,00248,7999.0,PEN,001102425000306973,P,2,NULL,NULL,NULL,LILIANA ELIZABETH AVENDAÐO CABRERA,17-Sep,MIV,NULL,NULL
9,2024-09-14,23980996,00248,5500.0,PEN,YOHANNI ELIZABETH MACEDA CRUZ,P,2,NULL,NULL,NULL,YOHANNI ELIZABETH MACEDA CRUZ,17-Sep,MIV,NULL,NULL


In [70]:
df_reactivas_multi = df_multi_final[(df_multi_final['TIPO_CARTERA'] == 'UNSECURED') & (df_multi_final['TIPO_FONDO'] != 'NULL')]
df_reactivas_multi

,FECHA,CC,CLAVSERV,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
5,2024-09-16,20808919,00248,500.0,PEN,40812376,P,2,001102999600065046,REACTIVA,KSTBC,IVAN EDQUEN ALTAMIRANO,17-Sep,MIV,UNSECURED,CLASA MORA
31,2024-09-16,28294750,00248,4000.0,PEN,GRUPO DE INVERSIONES EL BRILLO SAC,P,2,001107339600054646,REACTIVA,KSTBC,GRUPO DE INVERSIONESEL BRILLO SAC,17-Sep,MIV,UNSECURED,ASESCOM RJ


In [71]:
df_multi_final = df_multi_final[(df_multi_final['CONTRATO'] != 'NULL') & (df_multi_final['TIPO_CARTERA'] == 'UNSECURED') & (df_multi_final['TIPO_FONDO'] == 'NULL')]
df_multi_final.to_excel(multiproducto, index=False)

print(df_multi_final.shape)
df_multi_final.head(5)

(48, 16)


,FECHA,CC,CLAVSERV,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
0,2024-09-16,08434980,00248,360.5,PEN,YOLANDA ESPERANZA TONG RONDONA DE MEDINA,P,2,001107855003847218,NULL,KSTBC,YOLANDA ESPERANZA YONG RONDONA,17-Sep,MIV,UNSECURED,MORNESE MORA
1,2024-09-16,12322224,00248,1000.0,PEN,"PAREDES BRICE･O GENNO GUILLERMO,",P,2,001101199600155894,NULL,EXJ,GUILLERMO PAREDES BRICEÐO GENNO,17-Sep,MIV,UNSECURED,MORNESE MORA
3,2024-09-16,20597258,00248,1000.0,PEN,ANA MARIA LARA OTAZU 10340814,P,2,001102619600041700,NULL,EXJ,ANA MARIA LARA OTAZU,17-Sep,MIV,UNSECURED,ASESCOM RJ
4,2024-09-16,20756586,00248,1231.0,PEN,CLAVER SERGIO TITO QUISPE,P,2,001105045002540055,NULL,KSTBC,CLAVER SERGIO TITO QUISPE,17-Sep,MIV,UNSECURED,CLASA MORA
6,2024-09-16,21109140,00248,848.0,PEN,PEDRO ENIQUE VARGAS SAAVEDRA,P,2,001101835004201874,NULL,EXJ,PEDRO ENRIQUE VARGAS SAAVEDRA,17-Sep,MIV,UNSECURED,CLASA MORA


In [78]:
df_no_enviados

,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
0,2024-09-16,10569370,00248,338,11772.00,PEN,LILIANA GIOVANNA BERROCAL BERRIOS,P,,None,None,NaN,NaN,17-Sep,MIV,None,NaN
1,2024-09-16,20012314,00248,23,1075.00,PEN,CIRILA GAUDIOSA VERA ACOSTA DNI 07553798,P,,None,None,NaN,NaN,17-Sep,MIV,None,NaN
2,2024-09-16,20956913,00248,245,5000.00,PEN,JIMMY SERRANO VILLALTA.,P,,None,None,NaN,NaN,17-Sep,MIV,None,NaN
3,2024-09-16,21117543,00248,259,2665.23,PEN,KENY NIEVES HUAMANI JIMENEZ,P,,None,None,NaN,NaN,17-Sep,MIV,None,NaN
4,2024-09-16,21212725,00248,175,35813.16,PEN,JANET ESTHER LEIVA MEZA,P,1,001101629600205667,NULL,BC,JANET ESTHER LEIVA MEZA,17-Sep,MIV,SECURED,SIN AGENCIA
5,2024-09-14,21444076,00248,301,1800.00,PEN,PROVEEDOR DE SOLUCIONES SRL,P,1,001103049600331515,REACTIVA,KSTBC,PROVEEDOR DE SOLUCIONES SRLTDA,17-Sep,MIV,SECURED,SIN AGENCIA
6,2024-09-16,21547028,00248,295,271100.00,PEN,001102959600525037,P,1,001102959600525037,REACTIVA,BC,INCODIESEL SAC,17-Sep,MIV,SECURED,SIN AGENCIA
7,2024-09-16,23423292,00248,197,2000.00,PEN,HERALDO RAMOS DE LA CRUZ.,P,2,None,None,BC,HERALDO RAMOS DE LA CRUZ,17-Sep,MIV,None,SIN AGENCIA
8,2024-09-16,23842598,00248,109,9550.00,PEN,CONTRATISTAS MINEROS E & S E.I.R.L,P,1,001104845000698665,NULL,BC,CONTRATISTAS MINEROSE & S EIRL - CONTRAMIES EIRL,17-Sep,MIV,SECURED,SIN AGENCIA
9,2024-09-16,24539514,00248,174,16886.00,PEN,KAREN ELENA GUILLEN GARCIA,P,,None,None,NaN,NaN,17-Sep,MIV,None,NaN


In [72]:
df_no_enviados_final = pd.concat([df_no_enviados, df_no_enviados_multi])

df_no_enviados_final['CONTRATO'] = df_no_enviados_final['CONTRATO'].fillna('NULL')
df_no_enviados_final['TIPO_FONDO'] = df_no_enviados_final['TIPO_FONDO'].fillna('NULL')
df_no_enviados_final['CARTERA'] = df_no_enviados_final['CARTERA'].fillna('NULL')
df_no_enviados_final['TIPO_CARTERA'] = df_no_enviados_final['TIPO_CARTERA'].fillna('NULL')
df_no_enviados_final['NOMBRE_CLIENTE'] = df_no_enviados_final['NOMBRE_CLIENTE'].fillna('NULL')
df_no_enviados_final['AGENCIA'] = df_no_enviados_final['AGENCIA'].fillna('NULL')
df_no_enviados_final['CLAVSERV'] = df_no_enviados_final['CLAVSERV'].apply(lambda x: str(int(x)).zfill(5))

df_no_enviados_final.to_excel(no_enviados, index=False)

print(df_no_enviados_final.shape)
df_no_enviados_final.head(5)

(23, 17)


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
0,2024-09-16,10569370,00248,338.0,11772.00,PEN,LILIANA GIOVANNA BERROCAL BERRIOS,P,,NULL,NULL,NULL,NULL,17-Sep,MIV,NULL,NULL
1,2024-09-16,20012314,00248,23.0,1075.00,PEN,CIRILA GAUDIOSA VERA ACOSTA DNI 07553798,P,,NULL,NULL,NULL,NULL,17-Sep,MIV,NULL,NULL
2,2024-09-16,20956913,00248,245.0,5000.00,PEN,JIMMY SERRANO VILLALTA.,P,,NULL,NULL,NULL,NULL,17-Sep,MIV,NULL,NULL
3,2024-09-16,21117543,00248,259.0,2665.23,PEN,KENY NIEVES HUAMANI JIMENEZ,P,,NULL,NULL,NULL,NULL,17-Sep,MIV,NULL,NULL
4,2024-09-16,21212725,00248,175.0,35813.16,PEN,JANET ESTHER LEIVA MEZA,P,1,001101629600205667,NULL,BC,JANET ESTHER LEIVA MEZA,17-Sep,MIV,SECURED,SIN AGENCIA


In [73]:
df_reactiva_final = pd.concat([df_reactiva, df_reactivas_multi])
df_reactiva_final.to_excel(reactiva, index=False)

print(df_reactiva_final.shape)
df_reactiva_final.head(5)

(5, 17)


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
0,2024-09-16,25474777,00248,332.0,4000.0,PEN,JHENNY CARMEN RODRIGUEZ AMARO.,P,1,001101839600423847,REACTIVA,KSTBC,JHENNY CARMEN RODRIGUEZ AMARO,17-Sep,MIV,UNSECURED,MORNESE MORA
1,2024-09-14,29192668,00248,257.0,4419.0,PEN,PERCY MULLO ROJAS,P,1,001101119600222202,REACTIVA,KSTBC,PERCY MULLO ROJAS,17-Sep,MIV,UNSECURED,MORNESE MORA
2,2024-09-16,30487727,00248,162.0,43923.6,PEN,BRYAN ARTURO DAVILA GONZALEZ,P,1,001107649600097417,CONVENIO,EXJ,BRYAN ARTURO DAVILA GONZALEZ,17-Sep,MIV,UNSECURED,SIN AGENCIA
5,2024-09-16,20808919,00248,NaN,500.0,PEN,40812376,P,2,001102999600065046,REACTIVA,KSTBC,IVAN EDQUEN ALTAMIRANO,17-Sep,MIV,UNSECURED,CLASA MORA
31,2024-09-16,28294750,00248,NaN,4000.0,PEN,GRUPO DE INVERSIONES EL BRILLO SAC,P,2,001107339600054646,REACTIVA,KSTBC,GRUPO DE INVERSIONESEL BRILLO SAC,17-Sep,MIV,UNSECURED,ASESCOM RJ


In [74]:
df_enviados = pd.concat([df_mono, df_multi_final])

print(df_enviados.shape)
df_enviados.head(5)

(344, 16)


,FECHA,CC,CLAVSERV,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,TIPO_FONDO,CARTERA,AGENCIA
0,2024-09-16,01162268,00248,1700.0,PEN,ROSA LITA MANTILLA MORALES,P,1,001108319601385258,ROSA LITA MANTILLA MORALES,17-Sep,MIV,UNSECURED,NULL,EXJ,CLASA MORA
1,2024-09-16,04893700,00248,265.0,PEN,JOSE JULIO GARAY DUE･AS,P,1,001101405002103900,JOSE JULIO GARAY DUEÐAS,17-Sep,MIV,UNSECURED,NULL,KSTBC,MORNESE MORA
2,2024-09-16,05847052,00248,100.0,PEN,TARJETA,P,1,001105975000325936,MARIA VICTORIA PEREZ SOLF,17-Sep,MIV,UNSECURED,NULL,KSTBC,ASESCOM RJ
3,2024-09-16,07007612,00248,300.0,PEN,CARLOS ANTONIO TORRES FLORES 1,P,1,001101005001032297,CARLOS ANTONIO TORRES FLORES,17-Sep,MIV,UNSECURED,NULL,KSTBC,ASESCOM RJ
4,2024-09-16,07365110,00248,1000.0,PEN,",DOMINGO LIMA MAMANI",P,1,001102289600369463,DOMINGO LIMA MAMANI,17-Sep,MIV,UNSECURED,NULL,KSTBC,CLASA MORA


In [75]:
df_base_test_final = df_base_test[['CC', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE']]

df_base_test_final['KEY'] = df_base_test_final['CC'].astype(str) + df_base_test_final['IMPORTE'].astype(str) + df_base_test_final['MONEDA'] + df_base_test_final['NOMBRE']
df_enviados['KEY'] = df_enviados['CC'].astype(str) + df_enviados['IMPORTE'].astype(str) + df_enviados['MONEDA'] + df_enviados['NOMBRE']

df_enviados_final = df_enviados.merge(df_base_test_final[['KEY', 'CENTROPAGO']], on='KEY', how='left')
df_enviados_final.drop(columns=['KEY'], inplace=True)
df_enviados_final['CLAVSERV'] = df_enviados_final['CLAVSERV'].apply(lambda x: str(int(x)).zfill(5))
df_enviados_final.sort_values(by=['FLAG','CC'], inplace=True)
df_enviados_final.reset_index(drop=True, inplace=True)

cols_enviados = ['FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'AGENCIA']
df_enviados_final = df_enviados_final[cols_enviados]
df_enviados_final.to_excel(si_enviados, index=False)

print(df_enviados_final.shape)
df_enviados_final.head(5)

(344, 17)


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
0,2024-09-16,01162268,00248,118.0,1700.0,PEN,ROSA LITA MANTILLA MORALES,P,1,001108319601385258,NULL,EXJ,ROSA LITA MANTILLA MORALES,17-Sep,MIV,UNSECURED,CLASA MORA
1,2024-09-16,04893700,00248,960.0,265.0,PEN,JOSE JULIO GARAY DUE･AS,P,1,001101405002103900,NULL,KSTBC,JOSE JULIO GARAY DUEÐAS,17-Sep,MIV,UNSECURED,MORNESE MORA
2,2024-09-16,05847052,00248,597.0,100.0,PEN,TARJETA,P,1,001105975000325936,NULL,KSTBC,MARIA VICTORIA PEREZ SOLF,17-Sep,MIV,UNSECURED,ASESCOM RJ
3,2024-09-16,07007612,00248,190.0,300.0,PEN,CARLOS ANTONIO TORRES FLORES 1,P,1,001101005001032297,NULL,KSTBC,CARLOS ANTONIO TORRES FLORES,17-Sep,MIV,UNSECURED,ASESCOM RJ
4,2024-09-16,07365110,00248,228.0,1000.0,PEN,",DOMINGO LIMA MAMANI",P,1,001102289600369463,NULL,KSTBC,DOMINGO LIMA MAMANI,17-Sep,MIV,UNSECURED,CLASA MORA


In [76]:
format_excel(multi_path, "multi_final")
format_excel(react_path, "instruccion")
format_excel(no_enviados_path, "instruccion")
format_excel(enviados_path, "instruccion")

os.startfile(multi_path)
os.startfile(react_path)        
os.startfile(no_enviados_path)
os.startfile(enviados_path)